In [52]:
import pandas as pd
import numpy as np

In [53]:
companies_selected = pd.read_excel("Companies.xlsx")

In [54]:
# since TSR is in % we will grade wight in % as well
grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
companies_selected["GradeWeight"] = companies_selected["Grade"].map(grade_weight)

In [55]:
def calculate_grade_score(data, x):
    return (1 - x) * data["Weighted TSR"] + x * data["GradeWeight"]

In [56]:
# # default values set as: 16% for industry allocation and 4.5% mac allocation
# # beta value for potfolio should be below 1
# def monte_carlo_simulation(
#     data, max_industry_allocation=0.16, max_stock_allocation=0.045, target_beta=1.0
# ):

#     best_portfolio = None
#     best_portfolio_beta = float("inf")

#     for i in range(5000, 8000):
#         i = i / 10000
#         # we first want to normalise the TSR:
#         data["Industry TSR total"] = data.groupby("Industry")["TSR"].transform("sum")
#         data["weighted TSR"] = data["TSR"] / data["Industry TSR total"]
#         # This will give us cases where our grade is not the main factor and TSR is which provides us with different scores on multiple iterations
#         # execute the function mention above
#         data["Grade Score"] = data.apply(calculate_grade_score, axis=1, args=(i,))

#         # obtain a weight scross all the investments. Basically getting a ratio so values for the weights will be between 0 and 1
#         # basic normalisation
#         data["Weight"] = data["Grade Score"] / data["Grade Score"].sum()

#         # Lets focus on each industry
#         # what is the current weight allocated to each industry
#         data["Industry Allocation"] = data.groupby("Industry")["Weight"].transform(
#             "sum"
#         )
#         data["Allocation_by_restriction"] = np.where(
#             data["Industry Allocation"] > max_industry_allocation,
#             max_industry_allocation,
#             data["Industry Allocation"],
#         )

#         data["Normalised Weight"] = np.where(
#             data["Allocation_by_restriction"] == 0.16,
#             (data["Weight"] / data["Industry Allocation"]) * 0.16,
#             data["Weight"],
#         )

#         # Calculate Portfolio Beta
#         portfolio_beta = (data["Beta Value"] * data["Normalised Weight"]).sum()

#         if portfolio_beta < best_portfolio_beta:
#             best_run = i
#             target_beta = portfolio_beta
#             best_portfolio_investment = data
#             print(best_run, target_beta)

#     return best_run, target_beta, best_portfolio_investment

In [57]:
# FIGURE OUT:
# - As x increases, the objective functions get skewed to give more weightage to the 'GRADE WEIGHT' term
# - Also as x increases, the final beta value also keeps increasing (check output in the cell below), WHY?

In [68]:
####################################################################### ALTERNATE CODE
# default values set as: 16% for industry allocation and 4.5% mac allocation
# beta value for potfolio should be below 1
def monte_carlo_simulation(
    data, max_industry_allocation=0.16, max_stock_allocation=0.045, target_beta=1.0
):

    best_portfolio = None
    best_portfolio_beta = float("inf")

    for i in range(0, 10000):
        i = i / 10000
        # we first want to normalise the TSR:
        data["Industry TSR total"] = data.groupby("Industry")["TSR"].transform("sum")
        data["Weighted TSR"] = data["TSR"] / data["Industry TSR total"]
        # This will give us cases where our grade is not the main factor and TSR is which provides us with different scores on multiple iterations
        # execute the function mention above
        data["Grade Score"] = data.apply(calculate_grade_score, axis=1, args=(i,))

        # obtain a weight scross all the investments. Basically getting a ratio so values for the weights will be between 0 and 1
        # basic normalisation
        data["Weight"] = data["Grade Score"] / data["Grade Score"].sum()

        # Lets focus on each industry
        # what is the current weight allocated to each industry
        data["Industry Allocation"] = data.groupby("Industry")["Weight"].transform(
            "sum"
        )
        data["Allocation_by_restriction"] = np.where(
            data["Industry Allocation"] > max_industry_allocation,
            max_industry_allocation,
            data["Industry Allocation"],
        )

        data["Normalised Weight"] = np.where(
            data["Allocation_by_restriction"] == 0.16,
            (data["Weight"] / data["Industry Allocation"]) * 0.16,
            data["Weight"],
        )

        # Calculate Portfolio Beta
        portfolio_beta = (data["Beta Value"] * data["Normalised Weight"]).sum()

        if portfolio_beta < best_portfolio_beta:
            best_run = i * 10000
            target_beta = portfolio_beta
            best_portfolio_investment = data
            # print(best_run, target_beta)

    # print(data[["Industry", "Industry Allocation"]].drop_duplicates())
    # print(data[["Grade Score", "Weight"]].drop_duplicates())
    return best_run, target_beta, best_portfolio_investment

In [69]:
run_no, final_beta, ideal_portfolio = monte_carlo_simulation(companies_selected)
print(f"best run: {run_no} final_beta: {final_beta} ")
print(ideal_portfolio)

best run: 9999.0 final_beta: 0.9228180371851001 
                                Company Name          Industry  Beta Value  \
0                       NextEra Energy (NEE)         Utilities        0.49   
1                          Duke Energy (DUK)         Utilities        0.38   
2                         Atmos Energy (ATO)         Utilities        0.50   
3                             Avangrid (AGR)         Utilities        0.34   
4                          Otter Tail (OTTR)         Utilities        0.69   
..                                       ...               ...         ...   
74  COSTCO WHOLESALE CORPORATION (XNAS:COST)  Consumer Staples        0.79   
75      DOLLAR GENERAL CORPORATION (XNYS:DG)  Consumer Staples        0.45   
76       COLGATE-PALMOLIVE COMPANY (XNYS:CL)  Consumer Staples        0.41   
77            THE HERSHEY COMPANY (XNYS:HSY)  Consumer Staples        0.37   
78                    UNILEVER PLC (XBUE:UL)  Consumer Staples        0.23   

   Grade    TS

In [60]:
run_no

0.9999

In [61]:
final_beta

0.9228180371851001

In [62]:
ideal_portfolio

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Industry TSR total,Weighted TSR,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight
0,NextEra Energy (NEE),Utilities,0.49,B,53.47,80,358.64,0.149091,79.992015,0.012882,0.130435,0.130435,0.012882
1,Duke Energy (DUK),Utilities,0.38,C,57.51,70,358.64,0.160356,69.993016,0.011272,0.130435,0.130435,0.011272
2,Atmos Energy (ATO),Utilities,0.50,B,54.13,80,358.64,0.150931,79.992015,0.012882,0.130435,0.130435,0.012882
3,Avangrid (AGR),Utilities,0.34,A,-8.57,90,358.64,-0.023896,89.990998,0.014493,0.130435,0.130435,0.014493
4,Otter Tail (OTTR),Utilities,0.69,A,87.72,90,358.64,0.244591,89.991024,0.014493,0.130435,0.130435,0.014493
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,COSTCO WHOLESALE CORPORATION (XNAS:COST),Consumer Staples,0.79,D,17.43,60,177.69,0.098092,59.994010,0.009662,0.128824,0.128824,0.009662
75,DOLLAR GENERAL CORPORATION (XNYS:DG),Consumer Staples,0.45,B,14.09,80,177.69,0.079295,79.992008,0.012882,0.128824,0.128824,0.012882
76,COLGATE-PALMOLIVE COMPANY (XNYS:CL),Consumer Staples,0.41,A,15.40,90,177.69,0.086668,89.991009,0.014493,0.128824,0.128824,0.014493
77,THE HERSHEY COMPANY (XNYS:HSY),Consumer Staples,0.37,B,12.95,80,177.69,0.072880,79.992007,0.012882,0.128824,0.128824,0.012882
